#### Import

In [ ]:
import os
import time
from dotenv import load_dotenv

# Import LangChain components
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.chains import RetrievalQA


from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from langchain.chains import ConversationChain, LLMChain, SequentialChain
from pprint import pprint

from langchain_core.tools import Tool
from langchain.tools import tool
from langchain_experimental.utilities import PythonREPL
from langchain.agents import create_react_agent, AgentExecutor


#### Model

In [2]:
'''     Env setup and Gemini model initialization     '''
load_dotenv() 

# Check if the API key is loaded
if "GOOGLE_API_KEY" not in os.environ:
    print("Error: GOOGLE_API_KEY not found in environment variables.")
    exit()

# models: ['gemini-2.5-pro', 'gemma-3-27b-it', 'gemini-2.5-flash-lite]
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite", 
    temperature=0.7, 
)

#### Chat model

In [3]:
msg = llm.invoke(
  [
      SystemMessage(content="You are a supportive AI bot that suggests fitness activities to a user in one short sentence"),
      HumanMessage(content="I like high-intensity workouts, what should I do?"),
      AIMessage(content="You should try a CrossFit class"),
      HumanMessage(content="How often should I attend?")
  ]
)

In [5]:
print(msg)

content='You should aim for 3-5 CrossFit sessions per week, allowing for rest days.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--6168f114-2eee-4018-9cd5-47b4966ffe6e-0' usage_metadata={'input_tokens': 44, 'output_tokens': 18, 'total_tokens': 62, 'input_token_details': {'cache_read': 0}}


#### Prompt template

In [ ]:
"""    String prompt templates    """
prompt = PromptTemplate.from_template("Tell me one {adjective} joke about {topic}")
input_ = {"adjective": "funny", "topic": "cats"}

prompt.invoke(input_)

StringPromptValue(text='Tell me one funny joke about cats')

In [ ]:
"""    Chat prompt templates    """

# Create a ChatPromptTemplate with a list of message tuples
prompt = ChatPromptTemplate.from_messages([
 ("system", "You are a helpful assistant"),
 ("user", "Tell me a joke about {topic}")
])

# Create a dictionary with the variable to be inserted into the template      
input_ = {"topic": "cats"}


prompt.invoke(input_)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

In [ ]:
"""    MessagePlaceholder    """

# Create a ChatPromptTemplate with a system message and a placeholder for multiple messages
prompt = ChatPromptTemplate.from_messages([
("system", "You are a helpful assistant"),
MessagesPlaceholder("msgs")  # This will be replaced with one or more messages
])

# Create an input dictionary where the key matches the MessagesPlaceholder name
input_ = {"msgs": [HumanMessage(content="What is the day after Tuesday?")]}

prompt.invoke(input_)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the day after Tuesday?', additional_kwargs={}, response_metadata={})])

#### Output Parsers

"""    JSON    """


In [18]:
# Define your desired data structure (JSON) (LLM output).
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [22]:
# And a query intended to prompt a language model to populate the data structure.
joke_query = "Tell me a less common developer joke."

# Set up a parser + inject instructions into the prompt template.
output_parser = JsonOutputParser(pydantic_object=Joke)

# Get the formatting instructions for the output parser
# This generates guidance text that tells the LLM how to format its response
format_instructions = output_parser.get_format_instructions()

# Create a prompt template that includes:
# 1. Instructions for the LLM to answer the user's query
# 2. Format instructions to ensure the LLM returns properly structured data
# 3. The actual user query placeholder
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # Dynamic variables that will be provided when invoking the chain
    partial_variables={"format_instructions": format_instructions},  # Static variables set once when creating the prompt
)

# Create a processing chain that:
# 1. Formats the prompt using the template
# 2. Sends the formatted prompt to the Llama LLM
# 3. Parses the LLM's response using the output parser to extract structured data
chain = prompt | llm | output_parser

# Invoke the chain with a specific query about jokes
# This will:
# 1. Format the prompt with the joke query
# 2. Send it to the LLM
# 3. Parse the response into the structure defined by your output parser
# 4. Return the structured result
chain.invoke({"query": joke_query})

{'setup': 'Why do programmers prefer dark mode?',
 'punchline': 'Because light attracts bugs.'}

"""    CSV    """


In [ ]:
# Create an instance of the parser that will convert comma-separated text into a Python list
output_parser = CommaSeparatedListOutputParser()

# These instructions explain to the LLM that it should return items in a comma-separated format
format_instructions = output_parser.get_format_instructions()

# Create a prompt template that:
prompt = PromptTemplate(
    template="Answer the user query. {format_instructions}\nList five {subject}.",
    input_variables=["subject"],  # This variable will be provided when the chain is invoked
    partial_variables={"format_instructions": format_instructions},  # This variable is set once when creating the prompt
)

# Build a processing chain that:
chain = prompt | llm | output_parser

# Invoke the processing chain with "ice cream flavors" as the subject
# This will:
# 1. Substitute "ice cream flavors" into the prompt template
# 2. Send the formatted prompt to the LLM
# 3. Parse the LLM's comma-separated response into a Python list
chain.invoke({"subject": "ice cream flavors"})

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']

### Documents

`Document Object `

In [ ]:
# Create a Document instance with:
# 1. page_content: The actual text content about Python
# 2. metadata: A dictionary containing additional information about this document
Document(
    page_content="""Python is an interpreted high-level general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant indentation.""",
    metadata={
        'my_document_id' : 234234,                      # Unique identifier for this document
        'my_document_source' : "About Python",          # Source or title information
        'my_document_create_time' : 1680013019          # Unix timestamp for document creation (March 28, 2023)
    }
)

Document(metadata={'my_document_id': 234234, 'my_document_source': 'About Python', 'my_document_create_time': 1680013019}, page_content="Python is an interpreted high-level general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant indentation.")

`Document Loader`

In [56]:
# Create a PyPDFLoader instance by passing the URL of the PDF file
# The loader will download the PDF from the specified URL and prepare it for loading
loader = PyPDFLoader("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf")

# Call the load() method to:
# 1. Download the PDF if needed
# 2. Extract text from each page
# 3. Create a list of Document objects, one for each page of the PDF
# Each Document will contain the text content of a page and metadata including page number
document = loader.load()
document

[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-31T03:50:13+00:00', 'author': 'IEEE', 'moddate': '2023-12-31T03:52:06+00:00', 'title': 's8329 final', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content="* corresponding author - jkim72@kent.edu \nRevolutionizing Mental Health Care through \nLangChain: A Journey with a Large Language \nModel\nAditi Singh \n Computer Science  \n Cleveland State University  \n a.singh22@csuohio.edu \nAbul Ehtesham  \nThe Davey Tree Expert \nCompany  \nabul.ehtesham@davey.com \nSaifuddin Mahmud  \nComputer Science & \nInformation Systems  \n Bradley University  \nsmahmud@bradley.edu  \nJong-Hoon Kim* \n Computer Science,  \nKent State University,  \njkim72@kent.edu \nAbstract— Mental health challenges are on the rise in our \nmodern society, and the imperative to address mental di

In [31]:
print(document[2].page_content[:1000])  # print the page 2's first 1000 tokens

Figure 2. An AIMessage illustration 
C. Prompt Template 
Prompt templates [10] allow you to structure input for LLMs. 
They provide a convenient way to format user inputs and 
provide instructions to generate responses. Prompt templates 
help ensure that the LLM understands the desired context and 
produces relevant outputs. 
The prompt template classes in LangChain are built to 
make constructing prompts with dynamic inputs easier. Of 
these classes, the simplest is the PromptTemplate. 
D. Chain 
Chains [11] in LangChain refer to the combination of 
multiple components to achieve specific tasks. They provide 
a structured and modular approach to building language 
model applications. By combining different components, you 
can create chains that address various u se cases and 
requirements. Here are some advantages of using chains: 
• Modularity: Chains allow you to break down 
complex tasks into smaller, manageable 
components. Each component can be developed and 
tested independentl

`URL and website loader`

In [33]:
# Create a WebBaseLoader instance by passing the URL of the web page to load
# This URL points to the LangChain documentation's introduction page
loader = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")

# Call the load() method to:
# 1. Send an HTTP request to the specified URL
# 2. Download the HTML content
# 3. Parse the HTML to extract meaningful text
# 4. Create a list of Document objects containing the extracted content
web_data = loader.load()
web_data

[Document(metadata={'source': 'https://python.langchain.com/v0.2/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentA newer LangChain version is out! Check out the latest version.IntegrationsAPI referenceLatestLegacyMorePeopleContributingCookbooks3rd party tutorialsYouTubearXivv0.2Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangChain HubJS/TS Docs💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Q

In [ ]:
# Print the first 1000 characters of the page content from the first Document
# This provides a preview of the successfully loaded web content
# web_data[0] accesses the first Document in the list
# .page_content accesses the text content of that Document
# [:1000] slices the string to get only the first 1000 characters
print(web_data[0].page_content[:1000])

`Text splitters`: chunks should be created with some overlap to keep context between chunks.

In [74]:
# Create a CharacterTextSplitter with specific configuration:
# - chunk_size=200: Each chunk will contain approximately 200 characters
# - chunk_overlap=20: Consecutive chunks will overlap by 20 characters to maintain context
# - separator="\n": Text will be split at newline characters when possible
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20, separator="\n")

# Split the previously loaded document (PDF or other text) into chunks
# The split_documents method:
# 1. Takes a list of Document objects
# 2. Splits each document's content based on the configured parameters
# 3. Returns a new list of Document objects where each contains a chunk of text
# 4. Preserves the original metadata for each chunk
chunks = text_splitter.split_documents(document)

# Print the total number of chunks created
# This shows how many smaller Document objects were generated from the original document(s)
# The number depends on the original document length and the chunk_size setting
print(len(chunks))
# chunks

147


In [73]:
total_char = 0
for page in document:
    for token in page.page_content:
        # print(token, end='-')
        total_char += len(token)
    # print('\n\n\n<<===============================>>\n\n\n')
print(f"Chunks size approximation: {total_char / (200 - 20)}")

Chunks size approximation: 139.9611111111111


#### A

#### A

#### A